# Model prediction

In [ ]:
import os
import sys
from fnmatch import fnmatch

import numpy as np
import pandas as pd
pd.set_option("display.max_rows", 10) # ("display.max_rows", None)

import joblib

import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=UserWarning)

In [ ]:
# Collect datasets
path_in, pattern, datasets = "datasets/", "*.csv", []
for path, subdirs, files in os.walk(path_in):
    for name in files:
        if fnmatch(name, pattern):
            datasets += [os.path.join(path, name)]

for dataset in datasets:
    # Load dataset
    d = pd.read_csv(dataset)
    
    # Abort if dataset has 0 samples
    row_count, _ = d.shape
    if not row_count > 0:
        continue
    
    # Pick features
    # cols = list(d.columns.values)[1:-1]
    # print("\033[4m%s\033[0m\n\n%s\n\n" % (dataset, cols))
    
    key = dataset[29:-4]
    
    ### Aggregation relationships
    if "hulls" == key:
        fieldnames = ["ScaleX", "ScaleY", "ScaleZ",
                      "PositionX", "PositionY", "PositionZ",
                      "OrientationW", "OrientationX", "OrientationY", "OrientationZ",
                      "ObjectMass", "ObjectMaximumLife", "ObjectArmor",
                      "DestructionDelay", "EffectMultiplierKill", "EffectMultiplierDestroy",
                      "LocalDirectionX", "LocalDirectionY", "LocalDirectionZ"]
    elif "links" == key:
        fieldnames = ["DestructionDelay", "EffectMultiplierKill", "EffectMultiplierDestroy"]
    elif "weapons" == key:
        fieldnames = ["ScaleX", "ScaleY", "ScaleZ",
                      "PositionX", "PositionY", "PositionZ",
                      "OrientationW", "OrientationX", "OrientationY", "OrientationZ",
                      "ObjectMass", "ObjectMaximumLife", "ObjectArmor",
                      "DestructionDelay", "EffectMultiplierKill", "EffectMultiplierDestroy",
                      "LocalDirectionX", "LocalDirectionY", "LocalDirectionZ",
                      "ReloadTime", "AffectorScaleFactor", "AffectorPowerFactor",
                      "AffectorPositionX", "AffectorPositionY", "AffectorPositionZ"]
    elif "ai_units" == key:
        fieldnames = ["AIBehaviourDistance", "AIAttackSpeed"]
    elif "ai_movements" == key:
        fieldnames = ["Acceleration", "MinimumSpeed", "MaximumSpeed",
                      "ReachDestinationAtFullLinearSpeed", "AccelerationAngular",
                      "MaximumSpeedAngular", "ReachDestinationAtFullAngularSpeed"]
    
    ### Assocation relationships
    elif "links_hif_hull" == key:
        fieldnames = ["ScaleX", "ScaleY", "ScaleZ",
                      "PositionX", "PositionY", "PositionZ",
                      "OrientationW", "OrientationX", "OrientationY", "OrientationZ",
                      "ObjectMass", "ObjectMaximumLife", "ObjectArmor",
                      "LocalDirectionX", "LocalDirectionY", "LocalDirectionZ"]
    elif "links_his_hull" == key:
        fieldnames = ["ScaleX", "ScaleY", "ScaleZ",
                      "PositionX", "PositionY", "PositionZ",
                      "OrientationW", "OrientationX", "OrientationY", "OrientationZ",
                      "ObjectMass", "ObjectMaximumLife", "ObjectArmor",
                      "LocalDirectionX", "LocalDirectionY", "LocalDirectionZ"]
    elif "weapons_mobile_objects" == key:
        fieldnames = ["ScaleX", "ScaleY", "ScaleZ", "PositionX", "PositionY", "PositionZ",
                      "OrientationW", "OrientationX", "OrientationY", "OrientationZ",
                      "ObjectMass", "ObjectMaximumLife", "ObjectArmor",
                      "DestructionDelay", "EffectMultiplierKill", "EffectMultiplierDestroy",
                      "LocalDirectionX", "LocalDirectionY", "LocalDirectionZ",
                      "ReloadTime", "AffectorScaleFactor", "AffectorPowerFactor",
                      "AffectorPositionX", "AffectorPositionY", "AffectorPositionZ",
                      "LocalPositionX", "LocalPositionY", "LocalPositionZ"]
    elif "weapons_cannons" == key:
        fieldnames = ["ScaleX", "ScaleY", "ScaleZ", "PositionX", "PositionY", "PositionZ",
                      "OrientationW", "OrientationX", "OrientationY", "OrientationZ",
                      "ObjectMass", "ObjectMaximumLife", "ObjectArmor",
                      "DestructionDelay", "EffectMultiplierKill", "EffectMultiplierDestroy",
                      "LocalDirectionX", "LocalDirectionY", "LocalDirectionZ",
                      "ReloadTime", "AffectorScaleFactor", "AffectorPowerFactor",
                      "AffectorPositionX", "AffectorPositionY", "AffectorPositionZ",
                      "FiringTurnMaximum", "FiringTurnStart", "FiringTurnEnd", "Roll",
                      "FiringPositionX", "FiringPositionY", "FiringPositionZ",
                      "FiringDirectionX", "FiringDirectionY", "FiringDirectionZ"]

    print("\033[4m%s\033[0m\n\n%s\n\n" % (dataset, fieldnames))
    
    # Check there are no bad values in dataset
    # print(np.isnan(d[fieldnames]))
    
    # Load classifiers
    dtc = joblib.load(f"classifiers/{key}/DecisionTree.joblib")
    nvc = joblib.load(f"classifiers/{key}/DecisionTree.joblib")
    knc = joblib.load(f"classifiers/{key}/DecisionTree.joblib")
    
    # Classify
    pred_dtc = pd.Series(dtc.predict(d[fieldnames]))
    print("\033[1mDecisionTree\033[0m")
    print(pred_dtc, end="\n\n")
    
    pred_nvc = pd.Series(nvc.predict(d[fieldnames]))
    print("\033[1mNaiveBayes\033[0m")
    print(pred_nvc, end="\n\n")
    
    pred_knc = pd.Series(knc.predict(d[fieldnames]))
    print("\033[1mKNeighbors\033[0m")
    print(pred_knc, end="\n\n")